## Parsing unity VR log file using functions in the locproc module

In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

In [ ]:
# if you want to import files from the parent directory, please add the directory to the path:
#import sys, os, inspect
#currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
#parentdir = os.path.dirname(currentdir)
#sys.path.insert(0, os.path.dirname(parentdir))
#print(sys.path)

In [ ]:
from unityvr.preproc import logproc
from unityvr.viz import viz

In [ ]:
dirName = "../sample/"
fileName = 'Log_2021-07-18_17-32-12_testFeaturePos.json'

dat = logproc.openUnityLog(dirName, fileName)

for i in range(5):
    print(dat[i]['frame'])
    print(json.dumps(dat[i], sort_keys=True, indent=4))

### Construct data object from log file
#### Extract header info

In [ ]:
print(dat[0]['headerNotes'])
logproc.makeMetaDict(dat, fileName)

#### Pull out object positions as pandas dataframe

In [ ]:
objDf = logproc.objDfFromLog(dat)
objDf.head()

#### Pull out fly position as pandas dataframe
Position in VR

In [ ]:
posDf, ftDf, tsDf = logproc.timeseriesDfFromLog(dat)

In [ ]:
print(len(posDf))
posDf.head()

In [ ]:
posDf.angle.values

In [ ]:
# Extract ball radius
matching = [s for s in dat if "ficTracBallRadius" in s]
ballRadius = matching[0]["ficTracBallRadius"]
print(ballRadius) # in decimeters

#conversion factor to cm
dc2cm = 10

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(10,5), gridspec_kw={'width_ratios':[20,1]})
axs[0].plot(posDf.x*dc2cm,posDf.y*dc2cm,color='grey', linewidth=0.5)
cb = axs[0].scatter(posDf.x*dc2cm,posDf.y*dc2cm,s=5,c=posDf.angle, cmap='hsv')
axs[0].plot(posDf.x[0]*dc2cm,posDf.y[0]*dc2cm,'ok')
axs[0].text(posDf.x[0]*dc2cm+0.2,posDf.y[0]*dc2cm+0.2,'start')
axs[0].plot(posDf.x.values[-1]*dc2cm,posDf.y.values[-2]*dc2cm,'sk')
axs[0].set_aspect('equal')
axs[0].set_xlabel('x [cm]')
axs[0].set_ylabel('y [cm]')
viz.myAxisTheme(axs[0])
plt.colorbar(cb,cax=axs[1], label='head direction [degree]')

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(10,5), gridspec_kw={'width_ratios':[20,1]})
axs[0].plot(posDf.time,posDf.angle,color='grey', linewidth=0.5)
cb = axs[0].scatter(posDf.time,posDf.angle,s=5,c=posDf.angle, cmap='hsv')

In [ ]:
vt = np.hypot(posDf.dx*dc2cm, posDf.dy*dc2cm)/posDf.dt
vr = np.diff(posDf.angle*np.pi/180)/posDf.dt[1:]
fig, axs = plt.subplots(1,2,figsize=(10,4), gridspec_kw={'width_ratios':[1,1]})
axs[0].hist(vt,color='grey',bins=25, range=(0,5));
axs[0].set_xlabel('translational velocity [cm/s]')
axs[0].set_ylabel('count')
axs[1].hist(vr,color='grey',bins=25,range=(-6,6));
axs[1].set_xlabel('rotational velocity [rad/s]')
axs[1].set_ylabel('count')
for ax in axs:
    viz.myAxisTheme(ax)

Position based on Fictrac tracking

In [ ]:
print(len(ftDf))
ftDf.head()

Photodiode signal and frame rate dataframe

In [ ]:
print(len(tsDf))
tsDf.head()

In [ ]:
framesDf = tsDf[['frame','time','dt']].drop_duplicates().reset_index(level=0)[['frame','time','dt']]

In [ ]:
framesDf.head()

Fictrac updates come in at a higher rate than the VR frame rate. Fictrac can also only run for a subset of the VR run time and therefore the number of frames in fictrac and vr dataframes may not match.

#### Frame rate

In [ ]:
# Extract frame rate setting
matching = [s for s in dat if "refreshRateHz" in s]
forceframerate = matching[0]["refreshRateHz"]
print(forceframerate) # frame rate in Hz

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10,6))
ax[0].plot(framesDf.time, 1/framesDf.dt, 'k')
ax[1].plot(framesDf.frame, 1/framesDf.dt, 'k')
ax[0].set_xlabel('time [s]')
ax[1].set_xlabel('frame');
for i in range(2):
    ax[i].axhline(forceframerate,0,1,color='r', linewidth=0.7)
    ax[i].set_ylabel('frame rate [1/s]')
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10,6))
window = 200
for i, ts in enumerate([0,5000]):
    ax[i].plot(range(window),tsDf.pdsig[ts:ts+window],'-',color='grey',linewidth=0.5)
    
    even = tsDf[ts:ts+window][tsDf['frame'][ts:ts+window]%2==1]
    ax[i].plot(np.arange(window)[tsDf['frame'][ts:ts+window].values%2==1], even.pdsig, 'b.', label='even frame')
    
    odd = tsDf[ts:ts+window][tsDf['frame'][ts:ts+window]%2==0]
    ax[i].plot(np.arange(window)[tsDf['frame'][ts:ts+window].values%2==0], odd.pdsig, 'c.', label=
              'odd frame')
    
    ax[i].set_xlabel('time [s]')
    ax[i].set_ylabel('photo diode signal')
    ax[i].set_xlabel('sample');
ax[0].legend()
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,4))
for f in range(1000,1005):
    sig = tsDf.query('frame == '+str(float(f)))
    ax.plot(range(len(sig.pdsig)), sig.pdsig, '.-',  label='frame{}'.format(f))
ax.legend()

Compute frame rate from photodiode signal

In [ ]:
from scipy.signal import medfilt

tsDf["pdFilt"]  = tsDf.pdsig.values
tsDf.pdFilt[np.isfinite(tsDf.pdsig)] = medfilt(tsDf.pdsig[np.isfinite(tsDf.pdsig)])
tsDf["pdThresh"]  = 1*(np.asarray(tsDf.pdFilt>=0.2+np.nanmedian(tsDf.pdFilt.values)))

Generate interpolated time

In [ ]:
from scipy import interpolate

tsDf['framestart'] = np.hstack([0,1*np.diff(tsDf.time)>0])

tsDf['counts'] = 1
sampperframe = tsDf.groupby('frame').sum()[['time','dt','counts']].reset_index(level=0)
sampperframe['fs'] = sampperframe.counts/sampperframe.dt

frameStartIndx = np.hstack((0,np.where(tsDf.framestart)[0]))
frameStartIndx = np.hstack((frameStartIndx, frameStartIndx[-1]+sampperframe.counts.values[-1]-1))
frameIndx = tsDf.index.values

frameNums = tsDf.frame[frameStartIndx].values.astype('int')
frameNumsInterp = np.hstack((frameNums, frameNums[-1]+1))

timeAtFramestart = tsDf.time[frameStartIndx].values

#generate interpolated frames
frameinterp_f = interpolate.interp1d(frameStartIndx,frameNums)
tsDf['frameinterp'] = frameinterp_f(frameIndx)

timeinterp_f = interpolate.interp1d(frameStartIndx,timeAtFramestart)
tsDf['timeinterp'] = timeinterp_f(frameIndx)

In [ ]:
fig, ax = plt.subplots(4,1, figsize=(10,10))
ts = 1000
wind = 300
ax[0].plot(tsDf.timeinterp[ts:ts+wind], tsDf.pdsig[ts:ts+wind], 'k')
ax[0].plot(tsDf.timeinterp[ts:ts+wind], tsDf.pdFilt[ts:ts+wind], 'c')
ax[0].plot(tsDf.timeinterp[ts:ts+wind], tsDf.pdThresh[ts:ts+wind], '--',color='orange')
ax[0].set_ylabel('photodiode signal [V]')
ax[0].set_xlabel('time [s]')

pdChange = np.hstack((0,np.diff(tsDf.pdThresh)))
pdChangeT = tsDf.timeinterp.values[abs( pdChange )>0]

ax[1].plot(tsDf.timeinterp, tsDf.pdsig, 'k')
#ax[1].set_ylim(0,250)
ax[1].set_xlabel('time [s]')
ax[1].set_ylabel('photodiode signal [V]')

ax[2].plot(pdChangeT[1:], 1/np.diff(pdChangeT), '.', color='grey')
#ax[2].set_ylim(0,250)
ax[2].axhline(forceframerate,0,1,color='r', linewidth=0.7)
ax[2].set_xlabel('time [s]')
ax[2].set_ylabel('photodiode-based\nframe rate [1/s]')

ax[3].hist(1/np.diff(pdChangeT), 100, color='grey')
ax[3].set_xlim(0,250)
ax[3].axvline(forceframerate,0,1,color='r', linewidth=0.7)
ax[3].set_ylabel('count')
ax[3].set_xlabel('photodiode-based frame rate [1/s]')

fig.tight_layout()